In [70]:
from gurobipy import Model, GRB
import pandas as pd

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day1_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day1_3.csv")
problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day1_2.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day1_4.csv")

# Convert truck identifiers to string for consistency
demand_truck['truck_node_index'] = demand_truck['truck_node_index'].astype(str)
truck_nodes['index'] = truck_nodes['index'].astype(str)

# Extract costs and parameters
burrito_price = problem_data['burrito_price'][0]
ingredient_cost = problem_data['ingredient_cost'][0]
truck_cost = problem_data['truck_cost'][0]

# Create a Gurobi Model
model = Model("Optimal_Truck_Location")

# Decision Variables: Assign trucks to demand locations
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes['index'], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, 'scaled_demand']) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes['index']),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes['index']:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'demand_node_index'] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes['index']:
    if j in demand_truck['truck_node_index'].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'truck_node_index'] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes['index'])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, 'scaled_demand'] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, 'demand_node_index']: demand_truck.loc[i, 'truck_node_index']
        for i in demand_truck.index if x[i].x > 0.5
    }
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35 rows, 324 columns and 630 nonzeros
Model fingerprint: 0xcda84dfa
Variable types: 0 continuous, 324 integer (324 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -2850.000000
Presolve time: 0.00s
Presolved: 35 rows, 324 columns, 630 nonzeros
Variable types: 0 continuous, 324 integer (324 binary)

Root relaxation: objective 1.405000e+03, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1405.00000    0    9 -2850.0000 1405.00000   149%     -    0s
H    0     0               

In [90]:
from gurobipy import Model, GRB
import pandas as pd
import os

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_3.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_4.csv")

# Verify available files in the directory
folder_path = "/Users/pavanvinayveesam/downloads/"
print("Files in directory:", os.listdir(folder_path))

# Load problem data CSV, ensuring the correct file is used
try:
    problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_2.csv")
    print("Columns in problem_data before processing:", problem_data.columns)
except FileNotFoundError:
    raise FileNotFoundError("The problem_data.csv file is missing. Ensure the correct file is present in the directory.")

# Ensure 'burrito_price' column exists
if "burrito_price" not in problem_data.columns:
    raise KeyError(f"'burrito_price' column is missing! Available columns: {problem_data.columns}")

# Convert truck identifiers to string for consistency
demand_truck["truck_node_index"] = demand_truck["truck_node_index"].astype(str)
truck_nodes["index"] = truck_nodes["index"].astype(str)

# Extract cost parameters
burrito_price = problem_data["burrito_price"][0]
ingredient_cost = problem_data["ingredient_cost"][0]
truck_cost = problem_data["truck_cost"][0]

# Create a Gurobi model
model = Model("Optimal_Truck_Location")

# Decision Variables
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes["index"], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, "scaled_demand"]) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes["index"]),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes["index"]:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, "demand_node_index"] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes["index"]:
    if j in demand_truck["truck_node_index"].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, "truck_node_index"] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes["index"])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, "scaled_demand"] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, "demand_node_index"]: demand_truck.loc[i, "truck_node_index"]
        for i in demand_truck.index if x[i].x > 0.5
    }
    
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Files in directory: ['BAN 675-01_Text Mining_2025Spring-1.pdf', 'Lecture_pandas.ipynb', 'Bank OZK Resume.pdf', 'Budgeting in class(1).ipynb', 'Advt to class (5).ipynb', 'HW 3 (1).xlsx', 'summer_analyst_resume.docx', 'InstallLDBPackage64c-2-1-2-08.zip', 'BAN 630_Chap 7 notes (2).pdf', '1737353558199 (1).docx', 'Pavan Vinay Veesam - Banker Development Program Resume.pdf', '1741330488026.pdf', 'Rougir_Cosmetics_Solution.xlsx', 'MS_SQL_SERVER_Installation_on_Windows-1.docx', 'Install Respondus LockDown Browser (x64c) 663347111 2.pkg', 'CCFBank_Resume.docx', 'Associate Financial Analyst.pdf', 'PavanVinay_Financial_Analyst_Resume (10).pdf', 'BankScheduling (1).ipynb', 'ban612finalprojectsubmissiongroup3', 'BAN 630_Chap 6 (6 slides).pdf', 'Account_Analyst.pdf', 'GMT20241025-010048_Recording.transcript.vtt', 'round1-day1_problem_data.csv', 'annotated-Big%20Data%20Assignment%2001.docx.pdf', 'GMT20241025-010048_Recording.cc.vtt', 'TransMedics_Resume.docx', 'annotated-Case%204.docx.pdf', 'codebas

In [94]:
from gurobipy import Model, GRB
import pandas as pd
import os

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_3.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_4.csv")

# Verify available files in the directory
folder_path = "/Users/pavanvinayveesam/downloads/"
print("Files in directory:", os.listdir(folder_path))

# Load problem data CSV, ensuring the correct file is used
try:
    problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day2_2.csv")
    print("Columns in problem_data before processing:", problem_data.columns)
except FileNotFoundError:
    raise FileNotFoundError("The problem_data.csv file is missing. Ensure the correct file is present in the directory.")

# Ensure 'burrito_price' column exists
if "burrito_price" not in problem_data.columns:
    raise KeyError(f"'burrito_price' column is missing! Available columns: {problem_data.columns}")

# Convert truck identifiers to string for consistency
demand_truck["truck_node_index"] = demand_truck["truck_node_index"].astype(str)
truck_nodes["index"] = truck_nodes["index"].astype(str)

# Extract cost parameters
burrito_price = problem_data["burrito_price"][0]
ingredient_cost = problem_data["ingredient_cost"][0]
truck_cost = problem_data["truck_cost"][0]

# Create a Gurobi model
model = Model("Optimal_Truck_Location")

# Decision Variables
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes["index"], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, "scaled_demand"]) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes["index"]),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes["index"]:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, "demand_node_index"] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes["index"]:
    if j in demand_truck["truck_node_index"].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, "truck_node_index"] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes["index"])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, "scaled_demand"] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, "demand_node_index"]: demand_truck.loc[i, "truck_node_index"]
        for i in demand_truck.index if x[i].x > 0.5
    }
    
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Files in directory: ['BAN 675-01_Text Mining_2025Spring-1.pdf', 'Lecture_pandas.ipynb', 'Bank OZK Resume.pdf', 'Budgeting in class(1).ipynb', 'Advt to class (5).ipynb', 'HW 3 (1).xlsx', 'summer_analyst_resume.docx', 'InstallLDBPackage64c-2-1-2-08.zip', 'BAN 630_Chap 7 notes (2).pdf', '1737353558199 (1).docx', 'Pavan Vinay Veesam - Banker Development Program Resume.pdf', '1741330488026.pdf', 'Rougir_Cosmetics_Solution.xlsx', 'MS_SQL_SERVER_Installation_on_Windows-1.docx', 'Install Respondus LockDown Browser (x64c) 663347111 2.pkg', 'CCFBank_Resume.docx', 'Associate Financial Analyst.pdf', 'PavanVinay_Financial_Analyst_Resume (10).pdf', 'BankScheduling (1).ipynb', 'ban612finalprojectsubmissiongroup3', 'BAN 630_Chap 6 (6 slides).pdf', 'Account_Analyst.pdf', 'GMT20241025-010048_Recording.transcript.vtt', 'round1-day1_problem_data.csv', 'annotated-Big%20Data%20Assignment%2001.docx.pdf', 'GMT20241025-010048_Recording.cc.vtt', 'TransMedics_Resume.docx', 'annotated-Case%204.docx.pdf', 'codebas

In [98]:
from gurobipy import Model, GRB
import pandas as pd

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day3_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day3_3.csv")
problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day3_2.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day3_4.csv")

# Convert truck identifiers to string for consistency
demand_truck['truck_node_index'] = demand_truck['truck_node_index'].astype(str)
truck_nodes['index'] = truck_nodes['index'].astype(str)

# Extract costs and parameters
burrito_price = problem_data['burrito_price'][0]
ingredient_cost = problem_data['ingredient_cost'][0]
truck_cost = problem_data['truck_cost'][0]

# Create a Gurobi Model
model = Model("Optimal_Truck_Location")

# Decision Variables: Assign trucks to demand locations
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes['index'], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, 'scaled_demand']) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes['index']),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes['index']:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'demand_node_index'] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes['index']:
    if j in demand_truck['truck_node_index'].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'truck_node_index'] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes['index'])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, 'scaled_demand'] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, 'demand_node_index']: demand_truck.loc[i, 'truck_node_index']
        for i in demand_truck.index if x[i].x > 0.5
    }
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 884 columns and 1742 nonzeros
Model fingerprint: 0xa84f7909
Variable types: 0 continuous, 884 integer (884 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -3781.000000
Presolve time: 0.00s
Presolved: 59 rows, 884 columns, 1742 nonzeros
Variable types: 0 continuous, 884 integer (884 binary)

Root relaxation: objective 3.947000e+03, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3947.00000    0   15 -3781.0000 3947.00000   204%     -    0s
H    0     0             

In [102]:
from gurobipy import Model, GRB
import pandas as pd

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day4_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day4_3.csv")
problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day4_2.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day4_4.csv")

# Convert truck identifiers to string for consistency
demand_truck['truck_node_index'] = demand_truck['truck_node_index'].astype(str)
truck_nodes['index'] = truck_nodes['index'].astype(str)

# Extract costs and parameters
burrito_price = problem_data['burrito_price'][0]
ingredient_cost = problem_data['ingredient_cost'][0]
truck_cost = problem_data['truck_cost'][0]

# Create a Gurobi Model
model = Model("Optimal_Truck_Location")

# Decision Variables: Assign trucks to demand locations
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes['index'], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, 'scaled_demand']) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes['index']),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes['index']:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'demand_node_index'] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes['index']:
    if j in demand_truck['truck_node_index'].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'truck_node_index'] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes['index'])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, 'scaled_demand'] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, 'demand_node_index']: demand_truck.loc[i, 'truck_node_index']
        for i in demand_truck.index if x[i].x > 0.5
    }
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 59 rows, 884 columns and 1742 nonzeros
Model fingerprint: 0x36244d3d
Variable types: 0 continuous, 884 integer (884 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -4635.000000
Presolve time: 0.00s
Presolved: 59 rows, 884 columns, 1742 nonzeros
Variable types: 0 continuous, 884 integer (884 binary)

Root relaxation: objective 3.545000e+03, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3545.00000    0   15 -4635.0000 3545.00000   176%     -    0s
H    0     0             

In [104]:
from gurobipy import Model, GRB
import pandas as pd

# Load data from CSV files
demand_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day5_1.csv")
demand_truck = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day5_3.csv")
problem_data = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day5_2.csv")
truck_nodes = pd.read_csv(r"/Users/pavanvinayveesam/downloads/day5_4.csv")

# Convert truck identifiers to string for consistency
demand_truck['truck_node_index'] = demand_truck['truck_node_index'].astype(str)
truck_nodes['index'] = truck_nodes['index'].astype(str)

# Extract costs and parameters
burrito_price = problem_data['burrito_price'][0]
ingredient_cost = problem_data['ingredient_cost'][0]
truck_cost = problem_data['truck_cost'][0]

# Create a Gurobi Model
model = Model("Optimal_Truck_Location")

# Decision Variables: Assign trucks to demand locations
x = model.addVars(demand_truck.index, vtype=GRB.BINARY, name="assign")  # 1 if demand node is served by truck
y = model.addVars(truck_nodes['index'], vtype=GRB.BINARY, name="use_truck")  # 1 if truck is used

# Objective Function: Maximize Profit
model.setObjective(
    sum(x[i] * ((burrito_price - ingredient_cost) * demand_truck.loc[i, 'scaled_demand']) for i in demand_truck.index)
    - sum(y[j] * truck_cost for j in truck_nodes['index']),  # Truck cost applied only if the truck is used
    GRB.MAXIMIZE
)

# Constraints: Ensure each demand node is served exactly once
for demand in demand_nodes['index']:
    model.addConstr(
        sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'demand_node_index'] == demand) == 1, 
        name=f"demand_fulfillment_{demand}"
    )

# Ensure that a truck is marked as "used" if it's assigned to at least one demand node
for j in truck_nodes['index']:
    if j in demand_truck['truck_node_index'].values:  # Ensure truck exists in demand_truck dataset
        model.addConstr(
            sum(x[i] for i in demand_truck.index if demand_truck.loc[i, 'truck_node_index'] == j) <= y[j] * len(demand_nodes),
            name=f"truck_usage_{j}"
        )

# Optimize the Model
model.optimize()

# Print the Total Profit
if model.status == GRB.OPTIMAL:
    print(f"Total Profit: {model.ObjVal}")
    
    # Debugging Output: Check how many trucks are used
    total_trucks_used = sum(y[j].x for j in truck_nodes['index'])
    print(f"Total Trucks Used: {total_trucks_used}")
    
    # Check total demand met
    total_demand_met = sum(demand_truck.loc[i, 'scaled_demand'] for i in demand_truck.index if x[i].x > 0.5)
    print(f"Total Demand Met: {total_demand_met}")
    
    optimal_assignments = {
        demand_truck.loc[i, 'demand_node_index']: demand_truck.loc[i, 'truck_node_index']
        for i in demand_truck.index if x[i].x > 0.5
    }
    print("\nOptimal Truck Assignments:")
    for demand, truck in optimal_assignments.items():
        print(f"Demand Node {demand} is served by Truck {truck}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 3080 columns and 6104 nonzeros
Model fingerprint: 0x633c79fc
Variable types: 0 continuous, 3080 integer (3080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -6540.000000
Presolve time: 0.00s
Presolved: 110 rows, 3080 columns, 6104 nonzeros
Variable types: 0 continuous, 3080 integer (3080 binary)

Root relaxation: objective 1.127000e+04, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11270.0000    0   29 -6540.0000 11270.0000   272%     -    0s
H    0     0     